# Sequential Pattern Mining
## PrefixSpan Algorithm
PrefixSpan extracts frequent sequences with depth-first search **(DFS)** by executing SDB projection operations **`recursively`**.

In [1]:
import pandas as pd
import numpy as np
import sys
from time import time

### 1. Reading and Processing the Dataset

In [2]:
df = pd.read_csv("paths_finished_wiki.csv")
df.head(5)

,hashedIpAddress,timestamp,durationInSec,path,rating
0,6a3701d319fc3754,1297740409,166.0,14th_century;15th_century;16th_century;Pacific...,NaN
1,3824310e536af032,1344753412,88.0,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0
2,415612e93584d30e,1349298640,138.0,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN
3,64dd5cd342e3780c,1265613925,37.0,14th_century;Renaissance;Ancient_Greece;Greece,NaN
4,015245d773376aab,1366730828,175.0,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0


In [3]:
def data_preprocessing(df):
    res = []
    for path in df:
        elements = path.replace('<', '').replace('  ', ' ').lower().split(';') # separator
        tmp = []
        for element in elements:
            tmp.append(element.split())
        res.append(tmp)
    return res

In [4]:
data = data_preprocessing(df.path)

### 2. PrefixSpan Algorithm Implementation

**An overview of how the algorithm works**
<img src="imgs/prefix_span.png" width="600" height="300">

#### A simple example to follow along 
Note: this example exist in the slides of this subject

<img src="imgs/table1.png" width="400" height="200">

In [14]:
test_case = [[['a'], ['a', 'b', 'c'], ['a', 'c'], ['d'], ['c', 'f']],
 [['a', 'd'], ['c'], ['b', 'c'], ['a', 'e']],
 [['c', 'f'], ['a', 'b'], ['d', 'f'], ['c', 'b']],
 [['e', 'g'], ['a', 'f'], ['c'], ['b'], ['c']]]

sample_data = [
    ['C', 'A', 'G', 'A', 'A', 'G', 'T'],
    ['T', 'G', 'A', 'C', 'A', 'G'],
    ['G', 'A', 'A', 'G', 'T']
]

sample_data =[
[["a"], ["a", "b", "c"], ["a", "c"], ["c"]],
[["a"], ["c"], ["b", "c"]],
[["a", "b"], ["d"], ["c"], ["b"], ["c"]],
[["a"], ["c"], ["b"], ["c"]]
]

### 2.1. Find all repeating sequences by  Scanning  the SDB 
**There are 3 basic cases**:
1. The sequence begins with `simultaneously` elements together which we put all together in parentheses, then one of these elements is taken and placed with '**_**'.
2. The sequence begins with the '**_**', then the first element of the sequence is simply ignored and the next element is added to the pattern as follows in the next case.
3. The sequence starts up with a regular element (as is the case in our dataset), then the strings are scanned normally.

In [6]:
def calc_frequent_items(sdb, pattern, min_sup):
    if sdb is None:
         return []
    
    if len(pattern) > 0:
        last_element = pattern[-1]
        last_sub_element = last_element[-1]
        
    else:
        last_element = []
    
    freqs = {} 
    p_freqs = {} # p refers to elements -sub_elements- that in parentheses '()'.
    res = []
    for seq in sdb: 
        # case 1
        fseq = seq[0]
        at_beginning = True
        for sub_element in last_element:
            if sub_element not in fseq:
                at_beginning = False
        if at_beginning and len(last_element) > 0:
            pattern_limit = fseq.index(last_sub_element)
            if pattern_limit < len(fseq) - 1:
                for sub_element in fseq[pattern_limit + 1:]:
                    if sub_element in p_freqs.keys():
                        p_freqs[sub_element] += 1
                    else:
                        p_freqs[sub_element] = 1        
        #case 2
        if '_' in seq[0]:
            for sub_element in seq[0][1:]:
                if sub_element in p_freqs:
                    p_freqs[sub_element] += 1
                else:
                    p_freqs[sub_element] = 1
            seq = seq[1:]
            
        #case 3
        visited = [] 
        for element in seq:
            for sub_element in element:
                if sub_element not in visited:
                    visited.append(sub_element)
                    if sub_element in freqs.keys():
                        freqs[sub_element] += 1
                    else:
                        freqs[sub_element] = 1
                        
                        
    for item in freqs.items():
        if item[1] >= min_sup:
            res.append(([item[0]], item[1]))
    for item in p_freqs.items():
        if item[1] >= min_sup:
            res.append((['_',item[0]], item[1]))      
    return sorted(res, key=lambda p: item[0]) #lambda is a virtual function        

In [7]:
fi = calc_frequent_items(test_case,['a'] ,1)

In [8]:
fi

[(['a'], 4),
 (['b'], 4),
 (['c'], 4),
 (['d'], 3),
 (['f'], 3),
 (['e'], 2),
 (['g'], 1),
 (['_', 'd'], 1)]

### 2.2. Find subsets of sequential patterns
The subsets of sequential patterns can be mined by constructing corresponding projected databases and mine each `recursively`. The projected databases as well as sequential patterns found in them are listed in the following Table
<img src="imgs/table2.png" width="800" height="400">

In [9]:
def generate_SDB(sdb, pattern):
    res = []
    last_element = pattern[-1]
    last_sub_element = last_element[-1]
                        
    for seq in sdb:
        pp_sdb = [] # refers to partial proj. SDB
        
        for element in seq:
            # First Phase: check if the pattern 
            # check if the beginning of the sequence such like <(_ac)>.
            at_beginning = False
            if '_' in element:
                if len(last_element) > 1:
                    if last_element in element:
                        at_beginning = True
            # check if the beginning of the sequence is a normal case such like <ac>.
            else:
                at_beginning = True
                for sub_element in last_element:
                    if sub_element not in element:
                        at_beginning = False
                        break #check if the last element in pattern not at the end of the current element of the sequence
            if at_beginning:
                element_index = seq.index(element)
                pattern_limit_index = element.index(last_sub_element)
                if pattern_limit_index == len(element) - 1:
                    pp_sdb = seq[element_index + 1:]
                else:
                    pp_sdb = seq[element_index:]
                    celem = element[pattern_limit_index:]
                    celem[0] = '_'
                    pp_sdb[0] = celem
                break   
        if len(pp_sdb) != 0:
            res.append(pp_sdb)
    return res

In [10]:
# W = [[['a'],['b'],['c'],['a']]]
gs = generate_SDB(test_case, ['a'])
gs

[[['a', 'b', 'c'], ['a', 'c'], ['d'], ['c', 'f']],
 [['_', 'd'], ['c'], ['b', 'c'], ['a', 'e']],
 [['_', 'b'], ['d', 'f'], ['c', 'b']],
 [['_', 'f'], ['c'], ['b'], ['c']]]

#### A method to concat a new element with the current pattern

In [11]:
def concat_patterns(p1, p2, s1, s2):
    if p1[0][0] == '_':
            tmp = p1[0]
            p1[0].remove('_')
            p2[-1].extend(tmp)
            p2.extend(p1[1:])
    else:
            p2.extend(p1)
    return (p2, min(s2, s1))

### Based on above, the algorithm of PrefixSpan will be as follows.
**Input:** A sequence database `(SDB)`, a starting  values for the pattern and support (\[\], 100000) by deafult and the minimum support `(min_sup)`.

**Output:** The complete set of sequential patterns.

In [12]:
def prefix_span(sdb, pattern,sup, min_sup):
    patterns = []
    
    elements = calc_frequent_items(sdb, pattern, min_sup)
    for element in elements:
        
        p,sup = concat_patterns([element[0]],[pattern], element[1], sup) 
        patterns.append((p,sup))
        p_sdb = generate_SDB(sdb, p)
        p_patterns = prefix_span(p_sdb, p, sup, min_sup)
        patterns.extend(p_patterns)
    return patterns

#### A method to display the patterns and its support values

In [13]:
def print_results(res):    
    fnl_res = []
    for idx, pattern in enumerate(res):

        for element in pattern[0]:
            if len(element) == 0:
                    pattern[0].remove(element)
            for sub_element in element:
                if len(sub_element) == 0:
                    element.remove(sub_element)     
        fnl_res.append({
            'pattern' : str(pattern[0]).replace('[','').replace(']','').replace("\'",'').replace(',','->'),
            'support' : pattern[1]}) 

    return fnl_res

In [15]:
start_time = time()
# res=prefixSpan(data[:5000],[],10000,5)
res=prefix_span(sample_data,[],10000,2)
end_time = time()
elapsed = end_time - start_time
print('Elapsed time is %f seconds.' % elapsed)

Elapsed time is 0.000251 seconds.


In [16]:
print_results(res)

[{'pattern': 'a-> b', 'support': 4},
 {'pattern': 'a-> b-> b-> c', 'support': 4},
 {'pattern': 'a-> b-> b-> c-> c', 'support': 3},
 {'pattern': 'a-> b-> b-> c', 'support': 2},
 {'pattern': 'a-> b-> c', 'support': 4},
 {'pattern': 'a-> b-> c-> c', 'support': 4},
 {'pattern': 'a-> b-> c-> b', 'support': 3},
 {'pattern': 'a-> b-> c-> b-> c', 'support': 2},
 {'pattern': 'a-> b', 'support': 2},
 {'pattern': 'b-> c', 'support': 4},
 {'pattern': 'b-> c-> c', 'support': 3},
 {'pattern': 'b-> c-> c-> c', 'support': 2},
 {'pattern': 'b-> c', 'support': 2},
 {'pattern': 'c', 'support': 4},
 {'pattern': 'c-> c', 'support': 4},
 {'pattern': 'c-> b', 'support': 3},
 {'pattern': 'c-> b-> c', 'support': 2}]

In [38]:
start_time = time()
res=prefix_span(data[:5000],[],10000,5)
# res=prefix_span(test_case,[],10000,2)
end_time = time()
elapsed = end_time - start_time
print('Elapsed time is %f seconds.' % elapsed)

Elapsed time is 4.384469 seconds.


In [39]:
print_results(res)

[{'pattern': '14th_century', 'support': 141},
 {'pattern': '14th_century-> 15th_century', 'support': 15},
 {'pattern': '14th_century-> 15th_century-> 16th_century', 'support': 12},
 {'pattern': '14th_century-> 16th_century', 'support': 12},
 {'pattern': '14th_century-> atlantic_ocean', 'support': 6},
 {'pattern': '14th_century-> africa', 'support': 6},
 {'pattern': '14th_century-> africa-> atlantic_slave_trade', 'support': 6},
 {'pattern': '14th_century-> africa-> atlantic_slave_trade-> african_slave_trade',
  'support': 6},
 {'pattern': '14th_century-> africa-> african_slave_trade', 'support': 6},
 {'pattern': '14th_century-> atlantic_slave_trade', 'support': 6},
 {'pattern': '14th_century-> atlantic_slave_trade-> african_slave_trade',
  'support': 6},
 {'pattern': '14th_century-> african_slave_trade', 'support': 6},
 {'pattern': '14th_century-> europe', 'support': 6},
 {'pattern': '14th_century-> europe-> united_states', 'support': 6},
 {'pattern': '14th_century-> renaissance', 'supp

### Refrences

1. [Generalized Sequential Pattern Mining with Item Intervals](https://www.researchgate.net/publication/42803299_Generalized_Sequential_Pattern_Mining_with_Item_Intervals?enrichId=rgreq-59fcced5778bfc7cb6b286c585bbc9fa-XXX&enrichSource=Y292ZXJQYWdlOzQyODAzMjk5O0FTOjEzNDczNTY3MDAyNjI0MEAxNDA5MTM0ODk1MTA3&el=1_x_3&_esc=publicationCoverPdf)

2. [PrefixSpan: Mining Sequential Patterns Efficiently by Prefix-Projected Pattern Growth](https://ieeexplore.ieee.org/abstract/document/914830)